In [ ]:
import requests

BASE_URL = "https://px.hagstofa.is/pxen/api/v1/en"

def build_tree(url:str) -> dict:
    print(url)
    if url.endswith('.px'): # then it's a leaf
        return dict(leaf = True)
    
    response = requests.get(url)
    response.raise_for_status() 
    data = response.json()

    # Otherwise, it's a folder; recurse into it
    tree = {}
    try:
        for item in data:
            pathComponent = item.get('id') or item.get('dbid') 
            englishName = item.get("text")
            tree[pathComponent] = build_tree(f"{url}/{pathComponent}")
            tree[pathComponent]['englishName'] = englishName

    except KeyError:
        pass
    except KeyboardInterrupt:
        pass
    except Exception as e:
        print(e)
    finally:
        return tree

if __name__ == "__main__":
    api_tree = build_tree(BASE_URL)


https://px.hagstofa.is/pxen/api/v1/en
https://px.hagstofa.is/pxen/api/v1/en/Atvinnuvegir
https://px.hagstofa.is/pxen/api/v1/en/Atvinnuvegir/ferdathjonusta
https://px.hagstofa.is/pxen/api/v1/en/Atvinnuvegir/ferdathjonusta/farthegar
https://px.hagstofa.is/pxen/api/v1/en/Atvinnuvegir/ferdathjonusta/farthegar/1_farthegar
https://px.hagstofa.is/pxen/api/v1/en/Atvinnuvegir/ferdathjonusta/farthegar/1_farthegar/SAM02001.px
https://px.hagstofa.is/pxen/api/v1/en/Atvinnuvegir/ferdathjonusta/farthegar/1_farthegar/SAM02002.px
https://px.hagstofa.is/pxen/api/v1/en/Atvinnuvegir/ferdathjonusta/farthegar/1_farthegar/SAM02003.px
https://px.hagstofa.is/pxen/api/v1/en/Atvinnuvegir/ferdathjonusta/farthegar/2_eldraefni
https://px.hagstofa.is/pxen/api/v1/en/Atvinnuvegir/ferdathjonusta/farthegar/2_eldraefni/FER90001.px
https://px.hagstofa.is/pxen/api/v1/en/Atvinnuvegir/ferdathjonusta/farthegar/2_eldraefni/SAM02000.px
https://px.hagstofa.is/pxen/api/v1/en/Atvinnuvegir/ferdathjonusta/farthegar/2_eldraefni/SAM02

In [60]:
import json
json.pretty_print = True
def pretty_print(data):
    return json.dumps(data, indent=4, sort_keys=True)
print(pretty_print(api_tree))

{
    "Atvinnuvegir": {
        "englishName": "Atvinnuvegir",
        "ferdathjonusta": {
            "englishName": "Tourism",
            "farthegar": {
                "1_farthegar": {
                    "SAM02001.px": {
                        "englishName": "Passengers through Keflavik airport by citizenship and month 2002-2025",
                        "leaf": true
                    },
                    "SAM02002.px": {
                        "englishName": "Passengers from abroad on luxury liners at the Port of Reykjav\u00edk1984-2023",
                        "leaf": true
                    },
                    "SAM02003.px": {
                        "englishName": "Passengers and vehicles from abroad on car ferries 1981-2023",
                        "leaf": true
                    },
                    "englishName": "Passengers"
                },
                "2_eldraefni": {
                    "FER90001.px": {
                        "englishName": "Passen

In [84]:
import re
from urllib.parse import urljoin

pattern = re.compile(r"\d+\-\d+")

def toTypable(s:str) -> str:
    """Convert a string to a typable format by removing special characters and replacing spaces with underscores."""
    s = re.sub(r"[^a-zA-Z0-9_\-]", "_", s)
    s = re.sub(r"\s+", "_", s)
    s = re.sub(pattern, lambda m: m.group(0).replace("-", "_to_"), s)
    return s

class APINode:
    def __init__(self, pathComponent:str,parent:'APINode', children:dict=dict()):
        self.__pathComponent = pathComponent
        self.__children = children.copy() #shallow good
        self.__englishName = self.__children.pop('englishName', 'baseURL') 
        self.__parent = parent # can be None
        self.isLeaf = False # if it's a leaf, then it's a table

        # give it attributes by running through the children dict recusrively.
        # eg the url https://px.hagstofa.is/pxen/api/v1/en/Atvinnuvegir/ferdathjonusta/ferdaidnadurhagvisar
        # should be retrieved by obj.Atvinnuvegir.Tourism.Short_term_indicators_in_tourism
        # so englishName has a purpose.
        # obviously the latter is so much more readable than the former.
        # also user friendly
        self.__recurse()

    def __recurse(self):
        if self.__children.get('leaf'):
            self.isLeaf = True
            return
        
        for pathComponent, children in self.__children.items():
            tmp = APINode(pathComponent=pathComponent, parent=self, children=children)
            typableEnglishName = toTypable(tmp.__englishName)
            setattr(self, typableEnglishName, tmp)
    
    @property
    def str(self):
        if self.__parent:
            return f"{self.__parent.str}/{self.__pathComponent}"
        else:
            return self.__pathComponent

    def __str__(self):
        return self.str

    def __repr__(self):
        return f"<APINode {self}>"
    

    


In [85]:
from copy import deepcopy
test = deepcopy(api_tree)
n = APINode(pathComponent=BASE_URL, parent=None, children=test) 

In [87]:
n.Atvinnuvegir.Agriculture.Hunting.Salmon_caught_by_angling_rod_in_selected_rivers_1998_to_2021

<APINode https://px.hagstofa.is/pxen/api/v1/en/Atvinnuvegir/landbunadur/landveidi/LAN10301.px>

In [89]:
with open('../assets/endpointTree.json', 'w') as f:
    json.dump(api_tree, f, indent=4, sort_keys=True)

In [ ]:
def print_tree(tree, indent=0):
    for key, value in tree.items():
        print(" " * indent + str(key))
        if isinstance(value, dict):
            print_tree(value, indent + 2)

print_tree(api_tree)
 

Atvinnuvegir
  Tourism
    Passengers
      Passengers
        Passengers through Keflavik airport by citizenship and month 2002-2025
        Passengers from abroad on luxury liners at the Port of Reykjavík1984-2023
        Passengers and vehicles from abroad on car ferries 1981-2023
      Earlier tables
        Passengers through Keflavik airport by month 1995-2023
        Passengers and tourists through Keflavik Airport  2015-
        Passengers through Keflavik airport by citizenship and year 2003-2018
        Foreign passengers 1971-1999
    Short term indicators in tourism
      Aircraft movements at Keflavik airport 2017-2025 by Month and Flight Traffic
      Rental cars by registration and months 2010-2025
      VAT-turnover in activities related to tourism in Iceland 2016-
      Number of employees in activities related to tourism 2008-
    Travel survey
      Travel survey
        Number of trips by purpose, destination, gender and age 2021-2024
        Number of trips by purp

In [91]:
with open('../assets/endpointTree.json', 'r') as f:
    d = json.load(f)

n = APINode(pathComponent=BASE_URL, parent=None, children=d)

In [93]:
n.Atvinnuvegir.Agriculture.Live_stock_and_field_crops.Production_in_agriculture_from_1977

<APINode https://px.hagstofa.is/pxen/api/v1/en/Atvinnuvegir/landbunadur/landbufe/LAN10103.px>